In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
import os
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [2]:
labels = ['PNEUMONIA', 'NORMAL']
dataset = []
test_DS = []
val_DS = []

# Populate the dataset list for training

for label in labels:
    path = os.path.join("./Dataset/chest_xray/train", label)
    class_num = labels.index(label)
    for img_name in os.listdir(path):
        dataset.append({"img_name": os.path.join(label, img_name), "label": class_num})

# Populate the dataset list for testing

for label in labels:
    path = os.path.join("./Dataset/chest_xray/test", label)
    class_num = labels.index(label)
    for img_name in os.listdir(path):
        test_DS.append({"img_name": os.path.join(label, img_name), "label": class_num})

# Populate the dataset list for validation

for label in labels:
    path = os.path.join("./Dataset/chest_xray/val", label)
    class_num = labels.index(label)
    for img_name in os.listdir(path):
        val_DS.append({"img_name": os.path.join(label, img_name), "label": class_num})


In [3]:
train_df = pd.DataFrame(dataset)
test_df = pd.DataFrame(test_DS)
val_df = pd.DataFrame(val_DS)


In [4]:
img_size = 150
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor()
])

In [5]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.dataframe = annotation_file
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        img_name = os.path.join(self.root_dir, self.dataframe.iloc[index, 0])
        image = Image.open(img_name).convert("RGB")
        label = int(self.dataframe.iloc[index, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
train_dataset = CustomDataset(root_dir="./Dataset/chest_xray/train", annotation_file=train_df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = CustomDataset(root_dir="./Dataset/chest_xray/test", annotation_file=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

val_dataset = CustomDataset(root_dir="./Dataset/chest_xray/val", annotation_file=val_df, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [7]:
class ConvNeuralNetwork(nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=7, padding=3)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=11, padding=4)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self._initialize_fc_input_size(img_size)
        self.fc1 = nn.Linear(self.fc_input_size, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def _initialize_fc_input_size(self, img_size):
        # Dummy forward pass to calculate input size for fully connected layer
        x = torch.randn(1, 1, img_size, img_size)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.maxpool1(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.maxpool2(x)
        self.fc_input_size = x.view(x.size(0), -1).size(1)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.maxpool1(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.maxpool2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [8]:
batch_size = 32
num_classes = 2
learning_rate = 0.001
num_epochs = 1
model = ConvNeuralNetwork(num_classes)
model.load_state_dict(torch.load("lungs.pth"))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.005, momentum=0.9)


C:\Users\iamgo\AppData\Local\Temp\ipykernel_5956\1495722192.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("lungs.pth"))


In [9]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    n=0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        y_pred = model(inputs)
        loss = criterion(y_pred, labels)
        n=n+1
        print(f"{n}-loss:{loss.item():.4f}")
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    model.eval()
    with torch.no_grad():
        test_acc = 0
        count = 0
        for inputs, labels in test_loader:
            y_pred = model(inputs)
            test_acc += (torch.argmax(y_pred, 1) == labels).float().sum()
            count += len(labels)
        test_acc /= count

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Test Accuracy: {test_acc*100:.2f}%")


1-loss:0.0612
2-loss:0.1454
3-loss:0.0079
4-loss:0.0100
5-loss:0.2200
6-loss:0.2243
7-loss:0.1248
8-loss:0.0720
9-loss:0.0732
10-loss:0.0217
11-loss:0.0988
12-loss:0.0399
13-loss:0.0607
14-loss:0.0818
15-loss:0.0430
16-loss:0.0344
17-loss:0.0995
18-loss:0.0181
19-loss:0.0179
20-loss:0.0235
21-loss:0.1356
22-loss:0.0448
23-loss:0.1226
24-loss:0.1679
25-loss:0.0227
26-loss:0.0444
27-loss:0.0430
28-loss:0.1109
29-loss:0.1161
30-loss:0.1402
31-loss:0.1220
32-loss:0.0395
33-loss:0.0747
34-loss:0.0619
35-loss:0.0935
36-loss:0.0662
37-loss:0.0731
38-loss:0.0984
39-loss:0.1942
40-loss:0.1909
41-loss:0.0611
42-loss:0.0688
43-loss:0.1673
44-loss:0.0446
45-loss:0.0554
46-loss:0.0977
47-loss:0.2596
48-loss:0.1479
49-loss:0.0109
50-loss:0.0648
51-loss:0.0585
52-loss:0.0713
53-loss:0.0594
54-loss:0.0850
55-loss:0.0488
56-loss:0.3449
57-loss:0.0980
58-loss:0.1563
59-loss:0.1846
60-loss:0.0208
61-loss:0.0346
62-loss:0.0611
63-loss:0.0099
64-loss:0.0693
65-loss:0.1378
66-loss:0.0802
67-loss:0.0208
68-l

In [10]:
model.eval()
val_acc = 0
val_count = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        y_pred = model(inputs)
        
        # Get predicted and actual labels
        predicted_labels = torch.argmax(y_pred, 1)
        
        # Print predicted and actual values
        for pred, actual in zip(predicted_labels, labels):
            print(f"Predicted: {pred.item()}, Actual: {actual.item()}")
        
        val_acc += (predicted_labels == labels).float().sum()
        val_count += len(labels)
val_acc /= val_count
print(f"Validation Accuracy: {val_acc*100:.2f}%")

Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 0, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 1
Predicted: 0, Actual: 1
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Validation Accuracy: 81.25%


In [11]:
# Save model
torch.save(model.state_dict(), "lungs.pth")

In [12]:
import torch
img_s=(150,150)
bat_size=1
onnx_model_path='lungs.onnx'
model.eval()
sample_inp=torch.rand((bat_size,1,*img_s))

y=model(sample_inp)
torch.onnx.export(
    model,
    sample_inp,
    onnx_model_path,
    verbose=False,
    input_names=['input'],
    output_names=['output'],
    opset_version=12
)
print(f"Model has been converted to ONNX format and saved at {onnx_model_path}")

Model has been converted to ONNX format and saved at lungs.onnx


In [13]:
import onnx
from onnx_tf.backend import prepare


c:\Users\iamgo\anaconda3\envs\myenv\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [14]:
onnx_model_path='lungs.onnx'
tf_model_path='model_tf'
onnx_model=onnx.load(onnx_model_path)
tf_rep=prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

INFO:absl:Function `__call__` contains input name(s) x with unsupported characters which will be renamed to transpose_17_x in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: model_tf\assets


INFO:tensorflow:Assets written to: model_tf\assets
INFO:absl:Writing fingerprint to model_tf\fingerprint.pb


In [15]:
import tensorflow as tf
saved_model_dir='model_tf'
tflite_model_path='model.tflite'

In [16]:
converter=tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model=converter.convert()

In [17]:
with open(tflite_model_path,'wb') as f:
    f.write(tflite_model)

In [18]:
interpreter=tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()

input_shape=input_details[0]['shape']
input_data=np.array(np.random.random_sample(input_shape),dtype=np.float32)

interpreter.set_tensor(input_details[0]['index'],input_data)

interpreter.invoke()
output_data =interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[ 1.638462  -1.4357557]]


In [20]:
converter=tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)


In [ ]:
%pip install numpy

In [ ]:
%pip install matplotlib

In [ ]:
%pip install onnx


In [ ]:
%pip install tensorflow-addons

In [ ]:
!git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow && pip install -e .

In [ ]:
%pip install torch

In [ ]:
%pip install torchvision

In [ ]:
%pip install numpy

In [ ]:
%pip install onnx_tf